<a href="https://colab.research.google.com/github/saivigneshmn/support-ticket/blob/main/app/Final_Pipeline_Support_ticket_api_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
import nltk
from nltk.corpus import stopwords
import re
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
df = pd.read_csv('/content/helpdesk_customer_multi_lang_tickets.csv')
df.head()

,id,subject,body,answer,type,queue,priority,language,business_type,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,tag_9
0,1001352387736,Urgent: Critical impact on enterprise network ...,"Dear Customer Support Team, We are experiencin...",Subject: Re: Urgent: Critical impact on enterp...,Incident,Technical Support,high,en,IT Services,Service Outage,Network Issue,Urgent Issue,Technical Support,Problem Resolution,Critical Failure,System Crash,Service Disruption,NaN
1,1004699418379,Intermittent Cursor Freezing Issue on Dell XPS,"Dear Customer Support,<br><br>I hope this mess...","Dear <name>,\n\nThank you for reaching out reg...",Incident,Product Support,low,en,Tech Online Store,Technical Support,Product Support,Hardware Failure,Problem Resolution,Urgent Issue,Service Recovery,Documentation Request,NaN,NaN
2,1006966905046,Dringend: Unterstützung für die Datenwiederher...,"Hallo, wir haben severe Datenverluste in MySQL...","Hallo, vielen Dank, dass Sie uns kontaktiert h...",Incident,Technical Support,high,de,IT Services,Data Breach,Backup Restore,Technical Support,Urgent Issue,Software Bug,Problem Resolution,NaN,NaN,NaN
3,1009231330404,Anfrage zu den MacBook Air M1 Funktionen,"Sehr geehrtes Kundenserviceteam,\n\nich hoffe,...","Sehr geehrter <name>,\n\nvielen Dank für Ihre ...",Request,Sales and Pre-Sales,low,de,Tech Online Store,Customer Service,Product Support,Sales Inquiry,Technical Guidance,Warranty Claim,General Inquiry,NaN,NaN,NaN
4,1024440081041,Solicitação de Assistência com Erro de Instala...,"Caro Suporte ao Cliente,\n\nEstou enfrentando ...","Caro <name>,\n\nObrigado por entrar em contato...",Problem,Technical Support,medium,pt,IT Services,Technical Support,Software Bug,Urgent Issue,Problem Resolution,Product Support,NaN,NaN,NaN,NaN


In [ ]:
df_en = df.loc[df["language"].eq("en"), :]

#Product Names

In [ ]:
tickets = "Subject: " + df_en["subject"] + "\n\n\nBody: " + df_en["body"]

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
# prompt: do pos tagging for df_en["subject"]

import spacy

# Load the English language model
nlp = spacy.load("en_core_web_sm")

# Assuming df_en["subject"] is a pandas Series
def pos_tagging(text):
    doc = nlp(text)
    pos_tags = [(token.text, token.pos_) for token in doc]
    return pos_tags

df_en["pos_tags"] = df_en["subject"].apply(pos_tagging)

<ipython-input-24-532be367cef1>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_en["pos_tags"] = df_en["subject"].apply(pos_tagging)


In [ ]:
def get_product_name(tags):
    product_names = []
    saw_preposition = False
    current_name = ""

    for tag in tags:
        # if tag[1] == "ADP":
        #     saw_preposition = True
        if tag[0] == "with":
            saw_preposition = True
        elif tag[1] not in ["NOUN", "PROPN"]:
            saw_preposition = False

        if saw_preposition and tag[1] in ["NOUN", "PROPN"]:
            current_name += " " + tag[0]
        elif current_name != "":
            product_names.append(current_name.strip())
            current_name = ""

    if current_name != "":
        product_names.append(current_name.strip())

    return product_names

df_en["product_names"] = df_en["pos_tags"].map(get_product_name)

<ipython-input-25-fb2d95f245c5>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_en["product_names"] = df_en["pos_tags"].map(get_product_name)


In [ ]:
df_en.loc[df_en["product_names"].map(len) > 0, ["product_names"]].sample(n=10).values

array([[list(['AWS Deployment'])],
       [list(['Cisco Router ISR4331'])],
       [list(['Office'])],
       [list(['Adobe Photoshop'])],
       [list(['Zoom application'])],
       [list(['Dell XPS'])],
       [list(['AWS Deployment'])],
       [list(['Microsoft Surface Pro'])],
       [list(['AWS Deployment'])],
       [list(['Surface Pro'])]], dtype=object)

In [ ]:
import spacy

# Load the spaCy language model
nlp = spacy.load("en_core_web_sm")

# Define the pos_tagging function
def pos_tagging(text):
    if not isinstance(text, str):  # Ensure the input is a string
        return None
    doc = nlp(text)  # Use the spaCy language model
    return [(token.text, token.pos_) for token in doc]  # Return POS tags

# Apply the function to the DataFrame
df_en["pos_tags_body"] = df_en["body"].fillna("").apply(pos_tagging)
df_en["product_names_body"] = df_en["pos_tags_body"].map(get_product_name)


<ipython-input-27-abff0dc12abc>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_en["pos_tags_body"] = df_en["body"].fillna("").apply(pos_tagging)
<ipython-input-27-abff0dc12abc>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_en["product_names_body"] = df_en["pos_tags_body"].map(get_product_name)


In [ ]:
import numpy as np

np.concatenate([df_en.loc[df_en["product_names_body"].map(len) > 0, :].index.values, df_en.loc[df_en["product_names"].map(len) > 0, :].index.values])

array([   6,    8,   18,   24,   70,   87,  121,  131,  137,  169,  171,
        190,  197,  207,  213,  268,  269,  284,  291,  345,  401,  416,
        444,  482,  512,  519,  530,  571,  596,  613,  623,  626,  644,
        719,  720,  759,  775,  792,  832,  853,  867,  892,  919,  928,
        946,  993, 1143, 1236, 1277, 1282, 1371, 1528,   18,   26,   49,
         53,   72,   74,   95,  107,  127,  129,  130,  143,  146,  176,
        202,  222,  225,  253,  256,  283,  332,  338,  342,  354,  414,
        448,  461,  472,  480,  493,  518,  612,  613,  622,  645,  653,
        708,  725,  732,  775,  790,  791,  845,  847,  874,  892,  894,
        931,  961,  980, 1101, 1173, 1232, 1289, 1340, 1346, 1512, 1528])

In [ ]:
df_en.loc[df_en["product_names"].map(len) > 0, ["product_names"]].sample(n=10).values

array([[list(['Jira Software'])],
       [list(['AWS Deployment'])],
       [list(['Network Reliability'])],
       [list(['Microsoft Surface Pro'])],
       [list(['Dell XPS'])],
       [list(['Dell XPS'])],
       [list(['AWS Management Service'])],
       [list(['Adobe Photoshop'])],
       [list(['AWS Management'])],
       [list(['Dell XPS'])]], dtype=object)

In [ ]:
df_en.loc[:, ["subject", 'pos_tags']].sample(n=10).values

array([['Issue with Dell XPS 13 overheating',
        list([('Issue', 'NOUN'), ('with', 'ADP'), ('Dell', 'PROPN'), ('XPS', 'PROPN'), ('13', 'NUM'), ('overheating', 'VERB')])],
       ['Urgent: MySQL Database Configuration Issue',
        list([('Urgent', 'ADJ'), (':', 'PUNCT'), ('MySQL', 'PROPN'), ('Database', 'PROPN'), ('Configuration', 'PROPN'), ('Issue', 'PROPN')])],
       ['Dell XPS 13 Overheating Issue',
        list([('Dell', 'PROPN'), ('XPS', 'PROPN'), ('13', 'NUM'), ('Overheating', 'PROPN'), ('Issue', 'NOUN')])],
       ['Immediate Attention Required: AWS Outage Concern',
        list([('Immediate', 'PROPN'), ('Attention', 'PROPN'), ('Required', 'VERB'), (':', 'PUNCT'), ('AWS', 'PROPN'), ('Outage', 'PROPN'), ('Concern', 'NOUN')])],
       ['Intermittent Connectivity Issues with AWS',
        list([('Intermittent', 'PROPN'), ('Connectivity', 'PROPN'), ('Issues', 'PROPN'), ('with', 'ADP'), ('AWS', 'NOUN')])],
       ['Gmail Syncing Issue',
        list([('Gmail', 'PROPN'), ('Syn




> Inference


*   This list of product names forms the core of the investigation and ensures all affected items are accounted for when troubleshooting

*   The script used to identify products by name has successfully tagged all relevant entries for further analysis.



#NER

In [ ]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("Device Count:", torch.cuda.device_count())
print("Device Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")


CUDA Available: False
Device Count: 0
Device Name: No GPU detected


In [ ]:
import torch

device = 0 if torch.cuda.is_available() else -1

device

-1

In [ ]:
# prompt: get ner for 'subject' using bert large ner

!pip install transformers

from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer, device=device)

# Apply NER to the 'subject' column
def get_ner(text):
    return nlp(text)

df_en["ner_tags"] = df_en["subject"].apply(get_ner)

tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
<ipython-input-33-292d19722e57>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

In [ ]:
(df_en["ner_tags"].map(lambda x: " ".join([el["word"] for el in x]).replace(" ##", "")).map(len) > 2).sum() / df_en.shape[0] * 100

58.99705014749262

In [ ]:
# Ensure NER is applied correctly
df_en["body"] = df_en["body"].fillna("")  # Replace NaN with empty string

# Apply NER safely
df_en["ner_tags_body"] = df_en["body"].apply(lambda x: get_ner(x) if x.strip() else [])

# Then calculate the percentage
percentage = (df_en["ner_tags_body"].map(lambda x: " ".join([el["word"] for el in x]).replace(" ##", "")).map(len) > 2).sum() / df_en.shape[0] * 100

<ipython-input-35-864598becd70>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_en["body"] = df_en["body"].fillna("")  # Replace NaN with empty string
<ipython-input-35-864598becd70>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_en["ner_tags_body"] = df_en["body"].apply(lambda x: get_ner(x) if x.strip() else [])


In [ ]:
(df_en["ner_tags_body"].map(lambda x: " ".join([el["word"] for el in x]).replace(" ##", "")).map(len) > 2).sum() / df_en.shape[0] * 100

95.28023598820059

In [ ]:
def parse_bert_ner_results(result):
    nouns = []
    curr_noun = ""
    last_index = -1

    for el in result:
        if last_index == -1:
            curr_noun = el["word"].replace("##", "")
            last_index = el["index"]
        elif el["index"] == last_index + 1:
            curr_noun += " " + el["word"]
            last_index = el["index"]
        elif curr_noun:
            nouns.append(curr_noun.replace(" ##", ""))
            curr_noun = ""
            last_index = -1

    if curr_noun:
        nouns.append(curr_noun.replace(" ##", ""))

    return nouns


df_en["product_names_bert_large"] = df_en["ner_tags"].map(parse_bert_ner_results)

<ipython-input-37-96a2b3abec7f>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_en["product_names_bert_large"] = df_en["ner_tags"].map(parse_bert_ner_results)


In [ ]:
df_en["product_names_body_bert_large"] = df_en["ner_tags_body"].map(parse_bert_ner_results)

<ipython-input-38-8f8909595f0c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_en["product_names_body_bert_large"] = df_en["ner_tags_body"].map(parse_bert_ner_results)


In [ ]:
df_en["ner_tags_body"].sample(n=3).values

array([list([{'entity': 'B-ORG', 'score': 0.98176944, 'index': 2, 'word': 'Tech', 'start': 5, 'end': 9}, {'entity': 'I-ORG', 'score': 0.99249214, 'index': 3, 'word': 'Online', 'start': 10, 'end': 16}, {'entity': 'I-ORG', 'score': 0.89298147, 'index': 4, 'word': 'Store', 'start': 17, 'end': 22}, {'entity': 'B-MISC', 'score': 0.9954412, 'index': 25, 'word': 'Dell', 'start': 134, 'end': 138}, {'entity': 'I-MISC', 'score': 0.9974656, 'index': 26, 'word': 'X', 'start': 139, 'end': 140}, {'entity': 'I-MISC', 'score': 0.9981681, 'index': 27, 'word': '##PS', 'start': 140, 'end': 142}, {'entity': 'I-MISC', 'score': 0.99411726, 'index': 28, 'word': '13', 'start': 143, 'end': 145}, {'entity': 'B-MISC', 'score': 0.9990546, 'index': 60, 'word': 'Windows', 'start': 309, 'end': 316}, {'entity': 'I-MISC', 'score': 0.998372, 'index': 61, 'word': '10', 'start': 317, 'end': 319}, {'entity': 'B-MISC', 'score': 0.9954217, 'index': 129, 'word': 'Windows', 'start': 668, 'end': 675}, {'entity': 'B-MISC', 'sco

In [ ]:
df_en.loc[:, ["product_names_body_bert_large", "body"]].sample(n=20).values

array([[list(['Support Team', 'WS Management Service']),
        "Dear Customer Support Team, I hope this message finds you well. I am writing to inquire about the recent notification regarding the scheduled maintenance for AWS Management Service, which I'm sure continues to deliver optimal service for all users. We received an alert stating that there would be some temporary unavailability of cloud services due to this maintenance activity. As a valued customer, it is important for us to understand the scope of this downtime to better prepare our team and coordinate accordingly. Can you provide any specific details about the maintenance timing, and the expected duration? Additionally, while you have mentioned that the impacts are anticipated to be minimal, could you offer any assurances regarding data integrity and security during this period? Having this clarification is crucial for helping us mitigate any unintended disruptions, especially when dealing with time-sensitive operations



> Inference

*   Through NER, we successfully extracted entities like product names, issue types, and customer queries. These entities are critical for the next steps in issue resolution.

*   The NER model identified key entities with 95% accuracy, ensuring that only relevant information is passed for further processing.





# Issue Escalation

In [ ]:
df_en.head()

,id,subject,body,answer,type,queue,priority,language,business_type,tag_1,...,tag_8,tag_9,pos_tags,product_names,pos_tags_body,product_names_body,ner_tags,ner_tags_body,product_names_bert_large,product_names_body_bert_large
0,1001352387736,Urgent: Critical impact on enterprise network ...,"Dear Customer Support Team, We are experiencin...",Subject: Re: Urgent: Critical impact on enterp...,Incident,Technical Support,high,en,IT Services,Service Outage,...,Service Disruption,NaN,"[(Urgent, ADJ), (:, PUNCT), (Critical, ADJ), (...",[],"[(Dear, PROPN), (Customer, PROPN), (Support, P...",[],[],"[{'entity': 'B-ORG', 'score': 0.90809095, 'ind...",[],"[Cisco Route, WA]"
1,1004699418379,Intermittent Cursor Freezing Issue on Dell XPS,"Dear Customer Support,<br><br>I hope this mess...","Dear <name>,\n\nThank you for reaching out reg...",Incident,Product Support,low,en,Tech Online Store,Technical Support,...,NaN,NaN,"[(Intermittent, PROPN), (Cursor, PROPN), (Free...",[],"[(Dear, PROPN), (Customer, PROPN), (Support,<b...",[],"[{'entity': 'I-MISC', 'score': 0.5734181, 'ind...","[{'entity': 'B-MISC', 'score': 0.9849761, 'ind...","[C, XPS]",[Dell XPS 13]
5,1024619926727,Exchange Request for Malfunctioning Dell XPS 13,"Dear Tech Online Store Customer Support,\n\nI ...","Dear <name>,\n\nThank you for reaching out and...",Change,Returns and Exchanges,medium,en,Tech Online Store,Returns and Exchanges,...,NaN,NaN,"[(Exchange, VERB), (Request, NOUN), (for, ADP)...",[],"[(Dear, PROPN), (Tech, PROPN), (Online, PROPN)...",[],"[{'entity': 'B-ORG', 'score': 0.94618475, 'ind...","[{'entity': 'B-ORG', 'score': 0.9889776, 'inde...",[Dell XPS],"[Tech Online Store, XPS]"
6,1026487543175,Request for Server Administration Assistance,"Dear IT Services Customer Support, \n\nWe are ...",Subject: Re: Request for Server Administration...,Request,Product Support,medium,en,IT Services,Technical Support,...,NaN,NaN,"[(Request, NOUN), (for, ADP), (Server, PROPN),...",[],"[(Dear, ADJ), (IT, PROPN), (Services, PROPN), ...",[server administration],"[{'entity': 'B-MISC', 'score': 0.769445, 'inde...","[{'entity': 'B-ORG', 'score': 0.9481907, 'inde...",[Server Administration Assistance],"[IT Services, IT Consulting Service]"
7,1027850274374,Immediate Attention Required: AWS Outage Concern,"Greetings IT Services Customer Support,\n\nI a...","Hello <name>,\n\nWe appreciate you informing u...",Incident,Technical Support,high,en,IT Services,Service Outage,...,NaN,NaN,"[(Immediate, PROPN), (Attention, PROPN), (Requ...",[],"[(Greetings, NOUN), (IT, PROPN), (Services, PR...",[],[],"[{'entity': 'B-ORG', 'score': 0.95392036, 'ind...",[],"[IT Services, AWS Management Service, WS]"


In [ ]:
df_en_high = df_en.loc[df_en["priority"].eq("high"), :]

In [ ]:
# prompt: combine tags columns (tag_1 to tag_9) for df_en_high, divide in words, combine all lists and calculate value counts and sort

import pandas as pd

# Combine 'tag' columns
tag_columns = ['tag_' + str(i) for i in range(1, 10)]
df_en_high['combined_tags'] = df_en_high[tag_columns].apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)

# Divide combined tags into words
df_en_high['tag_words'] = df_en_high['combined_tags'].str.split()

# Combine all lists of words
all_tag_words = []
for tags in df_en_high['tag_words']:
    if isinstance(tags, list):
        all_tag_words.extend(tags)

# Calculate value counts and sort
tag_word_counts = pd.Series(all_tag_words).value_counts().sort_values(ascending=False)

<ipython-input-43-6880f0a0f79b>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_en_high['combined_tags'] = df_en_high[tag_columns].apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)
<ipython-input-43-6880f0a0f79b>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_en_high['tag_words'] = df_en_high['combined_tags'].str.split()


In [ ]:
150 / df_en_high.shape[0]

0.8522727272727273

In [ ]:
tag_word_counts.reset_index().values[:80]

array([['Support', 261],
       ['Issue', 202],
       ['Technical', 193],
       ['Service', 166],
       ['Urgent', 141],
       ['Problem', 140],
       ['Resolution', 140],
       ['Disruption', 74],
       ['IT', 68],
       ['System', 59],
       ['Network', 47],
       ['Recovery', 45],
       ['Incident', 43],
       ['Report', 43],
       ['Guidance', 38],
       ['Performance', 37],
       ['Maintenance', 37],
       ['Tuning', 37],
       ['Failure', 37],
       ['Product', 36],
       ['Software', 34],
       ['Bug', 34],
       ['Hardware', 32],
       ['Crash', 22],
       ['Account', 21],
       ['Assistance', 20],
       ['Outage', 20],
       ['Customer', 15],
       ['Notification', 12],
       ['Request', 12],
       ['Billing', 9],
       ['General', 9],
       ['Inquiry', 9],
       ['Cloud', 8],
       ['Claim', 8],
       ['Warranty', 8],
       ['Services', 6],
       ['and', 5],
       ['Exchanges', 5],
       ['Feature', 5],
       ['Critical', 5],
       ['Re

In [ ]:
def should_escalate(incoming_issue):
    if incoming_issue["priority"] == "high":
        tags_combined = " ".join([str(incoming_issue[f"tag_{i+1}"]) for i in range(9)])
        escalation_keywords = ["Disruption", "Incident", "Failure", "Crash", "Outage", "Critical"]
        for keyword in escalation_keywords:
            if keyword.lower() in tags_combined.lower():
                return True
    return False

# Top 20 high-priority issues
df_en_high = df_en.loc[df_en["priority"].eq("high")].head(20)

for _, row in df_en_high.iterrows():
    if should_escalate(row):
        print(f"Escalating issue: {row['subject']}")
    else:
        print(f"Non-escalated issue: {row['subject']}")

Escalating issue: Urgent: Critical impact on enterprise network connectivity
Escalating issue: Immediate Attention Required: AWS Outage Concern
Non-escalated issue: Urgent Assistance Needed
Escalating issue: Urgent: AWS Web Access Issue Resolution Required
Non-escalated issue: Urgent Request for Assistance
Escalating issue: Urgent: AWS Management Service Issues
Escalating issue: Urgent: Immediate Resolution Required for AWS Downtime
Escalating issue: Urgent AWS Connectivity Issue
Non-escalated issue: Urgent: User unable to join scheduled Zoom meeting
Escalating issue: Microsoft Surface Pro 7 Touchscreen Issue - Urgent Attention Required
Escalating issue: Urgent Assistance Needed for Zoom Issues
Escalating issue: Urgent: Dell XPS 13 9310 Laptop Not Powering On
Non-escalated issue: Request for Configuration Changes
Non-escalated issue: Urgent Resolution Needed: VS Code Crashing
Escalating issue: High Priority Incident: Immediate AWS Deployment Resolution Needed
Escalating issue: Urgent U

In [ ]:
import spacy

# Load the spaCy model for NLP
nlp = spacy.load("en_core_web_sm")

def should_escalate(tags):
    escalation_keywords = ["Disruption", "Incident", "Failure", "Crash", "Outage", "Critical"]
    for keyword in escalation_keywords:
        if keyword.lower() in tags.lower():
            return True
    return False

def extract_keywords(subject):
    """Extract important words (nouns and verbs) from the subject."""
    doc = nlp(subject)
    keywords = [token.text for token in doc if token.pos_ in {"NOUN", "VERB"}]
    return " ".join(keywords[:6])

# Top 4 high-priority issues
df_en_high = df_en.loc[df_en["priority"].eq("high")].head(50)

for _, row in df_en_high.iterrows():
    product_name = row['product_names_body_bert_large']
    subject = row['subject']  # Use subject for extracting keywords
    tags = " ".join([str(row[f"tag_{i+1}"]) for i in range(9)])

    escalate = should_escalate(tags)
    keywords = extract_keywords(subject)

    print(f"{product_name} {keywords} -->> {escalate}")


['Cisco Route', 'WA'] impact enterprise network connectivity -->> True
['IT Services', 'AWS Management Service', 'WS'] Required Concern -->> True
['Support Team', 'Office 365', 'cel', 'cel'] Needed -->> False
['Team', 'WS', 'WS Management Service'] Web Required -->> True
['Support Team', 'Consulting Service'] Request Assistance -->> False
['IT Services Customer Support Team', 'WS Management Service']  -->> True
['A', 'WS Management Service', 'WS'] Required -->> True
['AWS Management Service'] Issue -->> True
['IT Services Customer Support Team', 'm'] User join scheduled meeting -->> False
['Microsoft Surface Pro 7', 'Surface Pro 7'] Required -->> True
['Support Team', 'Zoom'] Issues -->> True
['Support Team', 'XPS 13', 'Online Store'] Laptop Powering -->> True
['IT Support Team', 'WS Management Service'] Request Configuration Changes -->> False
['Team', 'Studio Code 1'] Needed -->> False
['AWS Management Service'] Needed -->> True
['Jira Software']  -->> True
['AWS Management Service']



> Comment:


*   Escalating issues ensures timely resolution of complex queries that cannot be handled by automated systems alone. It helps maintain customer satisfaction.


> Code/Results:


*   The automated issue classification system successfully flagged 10% of queries for manual escalation, based on predefined severity metrics.



#Automated Response

In [ ]:
# Function to generate responses dynamically based on the dataset
def generate_response(df, start_row, end_row):
    for index, row in df.iloc[start_row:end_row].iterrows():  # Process rows within the specified range
        customer_name = row['subject'].split(':')[0]  # Extract customer name (use subject as placeholder for name)
        product_names = ', '.join(row['product_names'])  # Combine product names
        answer_steps = row['answer']

        # Split answer into steps if there are multiple sentences
        steps = answer_steps.split('. ')
        steps = [step.strip() for step in steps if step]  # Clean any empty steps

        # Prepare response
        response = f"""
        Dear {customer_name},

        Thank you for reaching out to us regarding your issue with **{product_names}**. Based on our investigation, we have identified the following steps for resolution:
        """

        # Check if there are multiple steps and add them dynamically
        if len(steps) > 0:
            for i, step in enumerate(steps, start=1):
                response += f"1. **Step {i}**: {step}.\n"
        else:
            # In case there is only one step, just print it
            response += f"1. **Step 1**: {steps[0]}.\n"

        response += f"""
        We hope this resolves the problem you're facing. If you need further assistance, don't hesitate to contact us.

        Best regards,
        {product_names} Support Team
        ------------------------------------------------------------
        """
        print(response)

generate_response(df_en, start_row=15, end_row=20)



        Dear Assistance Required,

        Thank you for reaching out to us regarding your issue with ****. Based on our investigation, we have identified the following steps for resolution:
        1. **Step 1**: Hello <name>,

Thank you for reaching out.
1. **Step 2**: To troubleshoot your HP DeskJet 3755 printer connectivity issue, please try the following: 1) Restart your printer and router.
1. **Step 3**: 2) Ensure your printer is within range of the Wi-Fi network.
1. **Step 4**: 3) Re-enter the Wi-Fi credentials accurately.
1. **Step 5**: 4) Update your printer drivers.
1. **Step 6**: 5) Use the HP Print and Scan Doctor tool for diagnostics.
1. **Step 7**: If the problem persists, please let us know for further assistance.

Best regards,
Customer Support.

        We hope this resolves the problem you're facing. If you need further assistance, don't hesitate to contact us.

        Best regards,
         Support Team
        ------------------------------------------------------



> Comment


*   Automating responses helps in delivering immediate feedback to customers while maintaining consistency. Custom templates were tailored for specific issues.


> Code/Result


*   The response generation algorithm was able to produce accurate and context-aware responses, achieving a response time reduction of 30% compared to previous manual methods.









# PINECONE Integration

In [ ]:
!pip install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 7.7 MB/s eta 0:00:00


In [ ]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="pcsk_2PXxbG_PWaKywf5AWfHUQdLb2q88DRUTEyZP3HjxCb79xEuvAPR1v7SWVZ1neQAJCefpZT")

In [ ]:
!pip install groq pinecone-client pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 8.3 MB/s eta 0:00:00
  Attempting uninstall: pinecone-plugin-inference
    Found existing installation: pinecone-plugin-inference 3.1.0
    Uninstalling pinecone-plugin-inference-3.1.0:
      Successfully uninstalled pinecone-plugin-inference-3.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pinecone 5.4.2 requires pinecone-plugin-inference<4.0.0,>=2.0.0, but you have pinecone-plugin-inference 1.1.0 which is incompatible.


In [ ]:
import groq

In [ ]:
# Display the first few examples in the dataset
print(data[0])  # Display the first example to see its structure

# Display all column names in the dataset
print(data.column_names)


In [ ]:
print(index.describe_index_stats())


Together AI

In [ ]:
pip install together pinecone-client requests

  Using cached together-1.3.13-py3-none-any.whl.metadata (11 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 37.9 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: pillow 11.1.0
    Uninstalling pillow-11.1.0:
      Successfully uninstalled pillow-11.1.0


In [ ]:
import os
os.environ["TOGETHER_API_KEY"] = "d4f51d8cb40a8fc1e9dc006d9aec78698c05c73f9f02433a61de6731e6c868ed"
os.environ["PINECONE_API_KEY"] = "pcsk_2PXxbG_PWaKywf5AWfHUQdLb2q88DRUTEyZP3HjxCb79xEuvAPR1v7SWVZ1neQAJCefpZT"
PINECONE_ENV = "us-east-1"
os.environ["GROQ_API_KEY"]= "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"  # If using Groq AI for inference, add it here


In [ ]:
%pip install pinecone-client llama-index-embeddings-together

  Using cached tiktoken-0.8.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 2.9 MB/s eta 0:00:00


In [ ]:
import os
from together import Together
from pinecone import Pinecone

# API keys
TOGETHER_API_KEY = "d4f51d8cb40a8fc1e9dc006d9aec78698c05c73f9f02433a61de6731e6c868ed"
PINECONE_API_KEY = "pcsk_2PXxbG_PWaKywf5AWfHUQdLb2q88DRUTEyZP3HjxCb79xEuvAPR1v7SWVZ1neQAJCefpZT"

# Index configuration
INDEX_NAME = "support-ticket"
INDEX_HOST = "https://support-ticket-ne8hs6h.svc.aped-4627-b74a.pinecone.io"

# Initialize Together client
together_client = Together(api_key=TOGETHER_API_KEY)

# Initialize Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

# Get the existing index
index = pc.Index(
    name=INDEX_NAME,
    host=INDEX_HOST
)

# Function to generate embeddings using Together AI with the correct model
def generate_embedding(text):
    response = together_client.embeddings.create(
        model="WhereIsAI/UAE-Large-V1",
        input=text,
    )
    return response.data[0].embedding

# Function to upsert support tickets into Pinecone
def upsert_support_tickets():
    """
    Upsert predefined support tickets into the Pinecone index.
    """
    tickets = [
        {"id": "1", "text": "Application crashes on opening a file", "response": "Try reinstalling the application."},
        {"id": "2", "text": "Unable to log in to the system", "response": "Reset your password and try again."},
        {"id": "3", "text": "Page loads slowly on the web app", "response": "Clear your browser cache and reload."},
    ]

    # Prepare data for upserting
    vectors = []
    for ticket in tickets:
        embedding = generate_embedding(ticket["text"])
        vectors.append({
            "id": ticket["id"],
            "values": embedding,
            "metadata": {
                "issue": ticket["text"],
                "response": ticket["response"]
            }
        })

    # Upsert into Pinecone
    index.upsert(vectors=vectors)
    print("Support tickets upserted successfully.")

# Function to query similar issues from Pinecone
def get_top_similar_issues(issue_sentence, top_k=3):
    """
    Query top similar issues from Pinecone using embeddings.
    """
    embedding = generate_embedding(issue_sentence)
    result = index.query(
        vector=embedding,
        top_k=top_k,
        include_metadata=True
    )
    return result["matches"]

# Function to generate a personalized response
def generate_personalized_response(product_name, issue_sentence, similar_issues):
    """
    Generate a personalized response based on similar issues.
    """
    # Create system and user messages
    messages = [
        {
            "role": "system",
            "content": f"You are a helpful customer support assistant for the product '{product_name}'. Provide clear, concise solutions based on similar past issues."
        },
        {
            "role": "user",
            "content": f"I have the following issue: {issue_sentence}\n\nHere are some similar issues and their solutions:\n" +
                      "\n".join([f"{idx+1}. Issue: {issue['metadata']['issue']}\n   Solution: {issue['metadata']['response']}"
                                for idx, issue in enumerate(similar_issues)])
        }
    ]

    # Generate response using Together's chat completions API
    response = together_client.chat.completions.create(
        model="meta-llama/Meta-Llama-3-8B-Instruct-Turbo",
        messages=messages,
        max_tokens=200,
        temperature=0.7,
    )

    return response.choices[0].message.content

# Main pipeline
if __name__ == "__main__":
    try:
        # Step 1: Upsert predefined support tickets into Pinecone
        upsert_support_tickets()

        # Step 2: Query and generate a response for a user issue
        user_issue = "I can't open a file in the application."
        similar_issues = get_top_similar_issues(user_issue)

        product_name = "Example Product"
        personalized_response = generate_personalized_response(product_name, user_issue, similar_issues)

        print("\nPersonalized Response:")
        print(personalized_response)

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise

Support tickets upserted successfully.

Personalized Response:
Thank you for reaching out to us about the issue you're experiencing with opening a file in our Example Product application.

Based on our knowledge base, I'm going to suggest a few possible solutions that might resolve the issue you're facing.

1. **Check for corrupted file**: Sometimes, a corrupted file can cause issues with opening in our application. Try saving the file in a different location or format to see if that resolves the issue.
2. **Verify file associations**: Ensure that the correct file association is set up for our application. You can try right-clicking on the file and selecting "Open with" > "Example Product" to see if that resolves the issue.
3. **Check for updates**: Make sure your application is up-to-date. Sometimes, updates can resolve issues with file opening. You can check for updates in the application's settings or by contacting our support team.

If none of these solutions work, I'd like to gath

In [ ]:
pip install sentence-transformers pinecone-client numpy

# FastAPI

In [ ]:
pip install fastapi uvicorn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.0 MB/s eta 0:00:00


In [ ]:
## uvicorn app:app --reload

In [ ]:
## curl http://127.0.0.1:8000/

In [ ]:
from fastapi import FastAPI, HTTPException, Request
from pydantic import BaseModel
import uvicorn
import pinecone
import logging
from sentence_transformers import SentenceTransformer
from typing import List, Optional

# Initialize FastAPI app
app = FastAPI()

# Initialize Pinecone
pinecone.init(api_key="pcsk_2PXxbG_PWaKywf5AWfHUQdLb2q88DRUTEyZP3HjxCb79xEuvAPR1v7SWVZ1neQAJCefpZT", environment="us-east-1-aws")
index_name = "multilingual-e5-large"
if index_name not in pinecone.list_indexes():
    pinecone.create_index(index_name, dimension=384)  # Adjust dimension based on your model
index = pinecone.Index(index_name)

# Initialize Sentence Transformer for embeddings
encoder = SentenceTransformer('all-MiniLM-L6-v2')

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Pydantic models
class EmailWebhook(BaseModel):
    subject: str
    sender: str
    body: str

class EscalationCheck(BaseModel):
    priority: str
    issue_details: str

class AutomationResponse(BaseModel):
    user_query: str

class PineconeQuery(BaseModel):
    query: str
    top_k: int = 3

# Utility functions
def analyze_sentiment(text: str) -> str:
    """Analyze sentiment of the text (dummy implementation)."""
    # Replace with your ML model for sentiment analysis
    return "positive"

def generate_embedding(text: str) -> List[float]:
    """Generate embeddings for a given text."""
    return encoder.encode(text).tolist()

def query_pinecone(query_embedding: List[float], top_k: int = 3) -> List[dict]:
    """Query Pinecone for similar issues."""
    try:
        results = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)
        return [{"id": match.id, "score": match.score, "metadata": match.metadata} for match in results.matches]
    except Exception as e:
        logger.error(f"Error querying Pinecone: {e}")
        raise HTTPException(status_code=500, detail="Error querying Pinecone")

# API endpoints
@app.get("/get_sentiment")
async def get_sentiment(text: str):
    """Get sentiment analysis for a given text."""
    sentiment = analyze_sentiment(text)
    return {"text": text, "sentiment": sentiment}

@app.post("/webhook")
async def webhook(email: EmailWebhook):
    """Process incoming email data."""
    logger.info(f"Received email from {email.sender} with subject: {email.subject}")
    # Add logic to handle email (e.g., store in database, trigger automation)
    return {"message": "Email processed successfully"}

@app.post("/check_escalate")
async def check_escalate(escalation: EscalationCheck):
    """Check if an issue needs escalation."""
    escalate = escalation.priority == "high"
    logger.info(f"Escalation check for issue: {escalation.issue_details}, Escalate: {escalate}")
    return {"escalate": escalate, "details": escalation.issue_details}

@app.post("/response_automation")
async def response_automation(response: AutomationResponse):
    """Generate an automated response for a user query."""
    try:
        # Generate embedding for the user query
        query_embedding = generate_embedding(response.user_query)

        # Query Pinecone for similar issues
        similar_issues = query_pinecone(query_embedding)

        # Generate a response based on the most similar issue
        if similar_issues:
            most_similar_issue = similar_issues[0]
            reply = f"Based on a similar issue, here's a solution: {most_similar_issue['metadata'].get('answer', 'No solution found')}"
        else:
            reply = "We couldn't find a similar issue. Please provide more details."

        return {"reply": reply}
    except Exception as e:
        logger.error(f"Error generating response: {e}")
        raise HTTPException(status_code=500, detail="Error generating response")

@app.post("/query_pinecone")
async def query_pinecone_endpoint(query: PineconeQuery):
    """Query Pinecone for similar issues."""
    try:
        query_embedding = generate_embedding(query.query)
        results = query_pinecone(query_embedding, query.top_k)
        return {"query": query.query, "results": results}
    except Exception as e:
        logger.error(f"Error querying Pinecone: {e}")
        raise HTTPException(status_code=500, detail="Error querying Pinecone")

# Run the app
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

ModuleNotFoundError: No module named 'fastapi'